# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686523


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:27,  2.90s/it]

Rendering models:   9%|▉         | 3/32 [00:07<01:13,  2.54s/it]

Rendering models:  12%|█▎        | 4/32 [00:08<00:54,  1.95s/it]

Rendering models:  16%|█▌        | 5/32 [00:08<00:40,  1.50s/it]

...

Rendering models:  88%|████████▊ | 28/32 [00:17<00:02,  1.95it/s]

Rendering models:  94%|█████████▍| 30/32 [00:18<00:00,  2.23it/s]

Rendering models:  97%|█████████▋| 31/32 [00:19<00:00,  2.21it/s]

Rendering models: 100%|██████████| 32/32 [00:19<00:00,  2.45it/s]

CamiloRuano                           0.006285
not-logged-in-68d3d82149e88f995533    0.004977
not-logged-in-bc5fdb9ea8d611e60b8b    0.001282
Almudys                               0.001457
not-logged-in-f7e2260ed4a9517a3e4a    0.002755
shirogane                             0.000601
asucich                               0.000491
austingasbarra                        0.000870
kjkavene                              0.000674
not-logged-in-c24ad414f260877715a6    0.000607
not-logged-in-2908c5b97527b9f7279b    0.001857
Roksolana                             0.000692
lattecuc                              0.002589
sn346808                              0.043261
sn346808                              0.034402
sn346808                              0.022090
LeeJoseph                             0.001021
not-logged-in-ffc5c906399855f5ef0f    0.000450
KrisKeogh                             0.001330
ayluminati                            0.002192
KevinKosa                             0.000645
plankjames   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())